# World Preview PoC

In [6]:
%pip install -qU nbformat kaleido pandas scipy plotly

You should consider upgrading via the '/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from scipy.special import comb

colors_drones = ['mediumblue','dodgerblue','dodgerblue','dodgerblue','dodgerblue','dodgerblue', 
                 'red','darkorange','darkorange','darkorange','darkorange','darkorange', 
                 'green','lime','lime','lime','lime','lime',
                 'yellow']

class Point:
  """
  A high-level construct to manipulate and debug 3D points easily.
  """
  def __init__(self, arr=[0,0,0], interest_level=0):
      """
      Default constructor.

      Args:
          arr: Array of 3D coordinates, x, y, and z respectively.
          interest_level: Optional interest level.
      """
      self.x = arr[0]
      self.y = arr[1]
      self.z = arr[2]
      self.interest = interest_level

  def __repr__(self):
      """
      Debug information contained in this structure.

      Returns:
          A construct that summarizes the properties in space of this
          Point instance.
      """
      return f'Point(x:{self.x}; y:{self.y}; z:{self.z}; int:{self.interest})'

def create_roi(x,y,z,size_x,size_y,size_z, color):
  xp = x + size_x
  yp = y + size_y
  zp = z + size_z

  return go.Mesh3d(
      x=[x, xp, xp, x, x, xp, xp, x],
      y=[y, y, yp, yp, y, y, yp, yp],
      z=[z, z, z, z, zp, zp, zp, zp],

      i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
      j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
      k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
      opacity=0.6,
      color=color
  )

def build_drone_trajectory(drone_positions, step=0.01):
    """
    Build drone trajectory using Bézier Curve model.

    Args:
        drone_positions: Positions of the drone assumed, as an array of Point(s).
        step: Optional step for detailed curve generation.

    Returns:
        Modelled trajectory as an array of Point(s).
    """
    # Interest = 0 => start/stop of a given trajectory, cut the Bézier Curve.
    trajectories = []
    trajectory = []

    # populate trajectories
    for pos in drone_positions:
        trajectory.append(pos)

        if pos.interest == 0 and len(trajectory) > 1:
            trajectories.append(trajectory)
            # clean up
            trajectory = [pos]

    # tolerance in case the user forgot to add the last point with interest = 0
    if len(trajectory) > 1:
        trajectories.append(trajectory)

    # construct trajectory
    B = []  # Bezier curve, our trajectory

    for tr in trajectories:
        for t in np.arange(0, 1, step):
            sum_x = 0.0
            sum_y = 0.0
            sum_z = 0.0
            n = len(tr) - 1

            for i in range(n + 1):
                a = comb(n, i, exact=True)
                b = (1 - t)**(n - i)
                c = t**i

                sum_x += a * b * c * tr[i].x
                sum_y += a * b * c * tr[i].y
                sum_z += a * b * c * tr[i].z

            B.append(Point([sum_x, sum_y, sum_z]))

    return B

def bezier(pois):
  proto = []

  # fill the proto
  for p in pois:
    l = p.interest if p.interest != 0 else 1
    for _ in range(l):
      proto.append(p)

  c = 0.001
  START = 0
  END = 1 + c
  STEP = 0.01

  B = [] # Bezier curve points

  for t in np.arange(START, END, STEP):
    sum_x = 0.0
    sum_y = 0.0
    sum_z = 0.0
    n = len(proto) - 1

    for i in range(n+1):  
      a = comb(n, i, exact=True)
      b = (1 - t)**(n - i)
      c = t**i

      sum_x += a * b * c * proto[i].x
      sum_y += a * b * c * proto[i].y
      sum_z += a * b * c * proto[i].z

    B.append(Point([sum_x,sum_y,sum_z]))

  return B

def split_trajectory(points):
  trs = []

  j = 0

  for i in range(len(points)):
    if i == 0:
      trs.append([points[i]])
    else:
      trs[j].append(points[i])

      if points[i].interest == 0:
        j += 1
        trs.append([points[i]])

  return trs

def create_curve(points, color='darkblue'):
  x = [p.x for p in points]
  y = [p.y for p in points]
  z = [p.z for p in points]

  return go.Scatter3d(
      x=x,
      y=y,
      z=z,
      marker=dict(
          size=0.1
      ),
      line=dict(
          color=color,
          width=1
      ),
      opacity=1
  )

def gen_trajectory(drones):
  tr_plots = []

  i = 0
  for d in drones:
    trajectories = split_trajectory(d)
    for tr in trajectories:
      curve = bezier(tr)
      plot = create_curve(curve, colors_drones[i] ) 
      tr_plots.append(plot)
    i += 1

  i = 0
  for d in drones:
    start = d[0]
    end = d[-1]
    if i in [0, 6, 12]:
        marker = 'square'
    elif i == 18:
        marker = 'diamond-open'
    else:
        marker = 'circle'
    tr_plots.append(go.Scatter3d(
        x=[start.x],
        y=[start.y],
        z=[start.z],
        marker_symbol = marker,
        marker_color=colors_drones[i], 
        opacity=1
    ))
    tr_plots.append(go.Scatter3d(
        x=[end.x],
        y=[end.y],
        z=[end.z],
        marker_symbol='x',
        marker_color=colors_drones[i], 
        opacity=0
    ))

    i += 1
  
  return tr_plots

def plot(rois, drones):
  fig = go.Figure(data=[*rois, *drones])
  fig.show()

In [6]:
rois = [
	create_roi(10, 256, 0, 88, 66, 28, 'lightgrey'),
	create_roi(268, 254, 0, 38, 66, 29, 'lightgrey'),
	create_roi(168, 254, 0, 82, 66, 24, 'lightgrey'),
	create_roi(114, 254, 0, 40, 68, 24, 'lightgrey'),
	create_roi(114, 172, 0, 38, 66, 29, 'lightgrey'),
	create_roi(8, 172, 0, 88, 68, 30, 'lightgrey'),
	create_roi(266, 170, 0, 38, 66, 25, 'lightgrey'),
	create_roi(168, 170, 0, 84, 66, 28, 'lightgrey'),
	create_roi(266, 90, 0, 38, 62, 30, 'lightgrey'),
	create_roi(168, 90, 0, 82, 64, 27, 'lightgrey'),
	create_roi(112, 90, 0, 38, 64, 24, 'lightgrey'),
	create_roi(6, 90, 0, 88, 64, 27, 'lightgrey'),
	create_roi(6, 12, 0, 144, 60, 24, 'lightgrey'),
	create_roi(262, 10, 0, 40, 62, 29, 'lightgrey'),
	create_roi(162, 10, 0, 88, 60, 28, 'lightgrey'),
]

drones = [
	[
		Point([80,100,50], 0)
	],
	[
		Point([110,74,1], 0)
	],
	[
		Point([55,157,25], 0)
	],
	[
		Point([45,85,1], 0)
	],
	[
		Point([42,155,1], 0)
	],
	[
		Point([39,85,10], 0)
	],
	[
		Point([100,240,50], 0)
	],
	[
		Point([64,254,1], 0)
	],
	[
		Point([73,254,5], 0)
	],
	[
		Point([100,254,1], 0)
	],
	[
		Point([68,246,1], 0)
	],
	[
		Point([100,285,10], 0)
	],
	[
		Point([230,140,50], 0)
	],
	[
		Point([260,98,20], 0)
	],
	[
		Point([217,160,10], 0)
	],
	[
		Point([293,156,1], 0)
	],
	[
		Point([212,80,10], 0)
	],
	[
		Point([260,205,20], 0)
	],
	[
		Point([60,120,40], 0)
	],
]
trs = gen_trajectory(drones)
fig = go.Figure(data=[*rois,*trs])
#fig.update_layout(scene_aspectmode='manual', scene_aspectratio=dict(x=1, y=1, z=0.5))
fig.update_layout(scene_aspectmode='data')
fig.show()